<a href="https://colab.research.google.com/github/groda/big_data/blob/master/Hadoop_Setting_up_a_Single_Node_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/groda/big_data"><div><img src="https://github.com/groda/big_data/blob/master/logo_bdb.png?raw=true" align=right width="90"></div></a>

# HDFS and MapReduce on a single-node Hadoop cluster
<br>
<br>

In this tutorial/notebook we'll showcase the setup of a single-node cluster, following the guidelines outlined on https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/SingleCluster.html. Subsequently, we'll demonstrate the seamless execution of elementary HDFS and MapReduce commands.

Upon downloading the software, several preliminary steps must be taken, including setting environment variables, generating SSH keys, and more. To streamline these tasks, we've consolidated them under the "Prologue" section.

Upon completion of the prologue, we can launch a single-node Hadoop cluster on the current virtual machine.

Following that, we'll execute a series of test HDFS commands and MapReduce jobs on the Hadoop cluster. These will be performed using a dataset sourced from a publicly available collection.

Finally, we'll proceed to shut down the cluster.


**TABLE OF CONTENTS**
* **[Prologue](#scrollTo=oUuQjW2oNMcJ)**

 * [Check the available Java version](#scrollTo=qFfOrktMPq8M)

 * [Download core Hadoop](#scrollTo=KE7kSYSXQYLf)

   * [Verify the downloaded file](#scrollTo=lGI4TNXPamMr)

 * [Configure `PATH`](#scrollTo=RlgP1ytnRtUK)

 * [Configure `core-site.xml` and `hdfs-site.xml`](#scrollTo=KLmxLQeJSb4A)

 * [Set environment variables](#scrollTo=kXbSKFyeMqr2)

 * [Setup localhost access via SSH key](#scrollTo=k2-Fdp73cF0V)

   * [Install `openssh` and start server](#scrollTo=-Uxmv3RdUwiF)

   * [Generate key](#scrollTo=PYKoSlaENuyG)

   * [Check SSH connection to localhost](#scrollTo=FwA6rKpScnVi)

* **[Launch a single-node Hadoop cluster](#scrollTo=V68C4cDySyek)**

   * [Initialize the namenode](#scrollTo=HTDPwnVlSbHS)

   * [Start cluster](#scrollTo=xMrEiLB_VAeR)

* **[Run some simple HDFS commands](#scrollTo=CKRRbwDFv3ZQ)**

* **[Run some simple MapReduce jobs](#scrollTo=G3KBe4R65bl1)**

   * [Simplest MapReduce job](#scrollTo=yVJA-3jSATGV)

   * [Another MapReduce example: filter a log file](#scrollTo=BbosNo0TD3oH)

   * [Aggregate data with MapReduce](#scrollTo=Sam22f-YT1xR)

* **[Stop cluster](#scrollTo=IF6-Z5RotAcO)**

* **[Concluding remarks](#scrollTo=w5N7tb0HSbZB)**



# Prologue

## Check the available Java version
 Apache Hadoop 3.3 and upper supports Java 8 and Java 11 (runtime only). See: https://cwiki.apache.org/confluence/display/HADOOP/Hadoop+Java+Versions


Check if Java version is one of `8`, `11`

In [1]:
!java -version

openjdk version "11.0.27" 2025-04-15
OpenJDK Runtime Environment Temurin-11.0.27+6 (build 11.0.27+6)
OpenJDK 64-Bit Server VM Temurin-11.0.27+6 (build 11.0.27+6, mixed mode)


In [2]:
%%bash
JAVA_MAJOR_VERSION=$(java -version 2>&1 | grep -m1 -Po '(\d+\.)+\d+' | cut -d '.' -f1)
if [[ $JAVA_MAJOR_VERSION -eq 8 || $JAVA_MAJOR_VERSION -eq 11 ]]
 then
 echo "Java version is one of 8, 11 ✓"
 fi

Java version is one of 8, 11 ✓


Find the variable for the environment variable `JAVA_HOME`

Find the path for the environment variable `JAVA_HOME`

In [3]:
!readlink -f $(which java)

/usr/lib/jvm/temurin-11-jdk-amd64/bin/java


Extract JAVA_HOME from the Java path by removing the `bin/java` part in the end

In [4]:
%%bash
JAVA_HOME=$(readlink -f $(which java) | sed 's/\/bin\/java$//')
echo $JAVA_HOME

/usr/lib/jvm/temurin-11-jdk-amd64


## Download core Hadoop
Download the latest stable version of the core Hadoop distribution from one of the download mirrors locations https://www.apache.org/dyn/closer.cgi/hadoop/common/.

**Note** with the option `--no-clobber`, `wget` will not download the file if it already exists.

In [5]:
!wget --no-clobber https://dlcdn.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz

--2025-07-12 14:55:28--  https://dlcdn.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz


Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.


HTTP request sent, awaiting response... 

200 OK
Length: 965537117 (921M) [application/x-gzip]
Saving to: ‘hadoop-3.4.0.tar.gz’

hadoop-3.4.0.tar.gz   0%[                    ]       0  --.-KB/s               

hadoop-3.4.0.tar.gz   5%[>                   ]  55.16M   276MB/s               

...

hadoop-3.4.0.tar.gz  79%[==============>     ] 728.42M   303MB/s               

hadoop-3.4.0.tar.gz  85%[================>   ] 791.73M   304MB/s               

hadoop-3.4.0.tar.gz  92%[=================>  ] 848.50M   303MB/s               

hadoop-3.4.0.tar.gz  98%[==================> ] 908.92M   303MB/s    eta 0s     

hadoop-3.4.0.tar.gz 100%[===================>] 920.81M   302MB/s    in 3.0s    

2025-07-12 14:56:02 (302 MB/s) - ‘hadoop-3.4.0.tar.gz’ saved [965537117/965537117]



Uncompress archive

In [6]:
%%bash
if [ ! -d "hadoop-3.4.0" ]; then
  tar xzf hadoop-3.4.0.tar.gz
fi

### Verify the downloaded file

(see https://www.apache.org/dyn/closer.cgi/hadoop/common/)

Download sha512 file

In [7]:
! wget --no-clobber https://dlcdn.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz.sha512

--2025-07-12 14:56:11--  https://dlcdn.apache.org/hadoop/common/hadoop-3.4.0/hadoop-3.4.0.tar.gz.sha512
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 160 [text/plain]
Saving to: ‘hadoop-3.4.0.tar.gz.sha512’

hadoop-3.4.0.tar.gz 100%[===================>]     160  --.-KB/s    in 0s      

2025-07-12 14:56:12 (4.89 MB/s) - ‘hadoop-3.4.0.tar.gz.sha512’ saved [160/160]



Compare

In [8]:
%%bash
A=$(sha512sum hadoop-3.4.0.tar.gz | cut - -d' ' -f1)
B=$(cut hadoop-3.4.0.tar.gz.sha512 -d' ' -f4)
printf "%s\n%s\n" $A $B
[[ $A == $B ]] && echo "True"

6f653c0109f97430047bd3677c50da7c8a2809d153b231794cf980b3208a6b4beff8ff1a03a01094299d459a3a37a3fe16731629987165d71f328657dbf2f24c


6f653c0109f97430047bd3677c50da7c8a2809d153b231794cf980b3208a6b4beff8ff1a03a01094299d459a3a37a3fe16731629987165d71f328657dbf2f24c


True


## Configure `PATH`

Add the Hadoop folder to the `PATH` environment variable


In [9]:
!echo $PATH

/opt/hostedtoolcache/Java_Temurin-Hotspot_jdk/11.0.27-6/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64:/snap/bin:/home/runner/.local/bin:/opt/pipx_bin:/home/runner/.cargo/bin:/home/runner/.config/composer/vendor/bin:/usr/local/.ghcup/bin:/home/runner/.dotnet/tools:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


In [10]:
import os
os.environ['HADOOP_HOME'] = os.path.join(os.getcwd(), 'hadoop-3.4.0')
os.environ['PATH'] = ':'.join([os.path.join(os.environ['HADOOP_HOME'], 'bin'), os.environ['PATH']])
#os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [11]:
import os
print(os.environ)

environ({'LANG': 'C.UTF-8', 'PATH': '/home/runner/work/big_data/big_data/hadoop-3.4.0/bin:/opt/hostedtoolcache/Java_Temurin-Hotspot_jdk/11.0.27-6/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64:/snap/bin:/home/runner/.local/bin:/opt/pipx_bin:/home/runner/.cargo/bin:/home/runner/.config/composer/vendor/bin:/usr/local/.ghcup/bin:/home/runner/.dotnet/tools:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'MAIL': '/var/mail/root', 'LOGNAME': 'root', 'USER': 'root', 'HOME': '/root', 'SHELL': '/bin/bash', 'TERM': 'xterm-color', 'SUDO_COMMAND': '/usr/bin/env PATH=/opt/hostedtoolcache/Java_Temurin-Hotspot_jdk/11.0.27-6/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64:/snap/bin:/home/runner/.local/bin:/opt/pipx_bin:/home/runner/.cargo/bin:/home/runner/.config/composer/vendor/bin:/usr/local/.ghcup/bin:/home/runner/.dotnet/tools:/usr/local/sbin:/usr/local/bin:/usr

In [12]:
!echo $PATH

/home/runner/work/big_data/big_data/hadoop-3.4.0/bin:/opt/hostedtoolcache/Java_Temurin-Hotspot_jdk/11.0.27-6/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64/bin:/opt/hostedtoolcache/Python/3.8.18/x64:/snap/bin:/home/runner/.local/bin:/opt/pipx_bin:/home/runner/.cargo/bin:/home/runner/.config/composer/vendor/bin:/usr/local/.ghcup/bin:/home/runner/.dotnet/tools:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin


## Configure `core-site.xml` and `hdfs-site.xml`

Edit the file `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml` to configure pseudo-distributed operation.

**`etc/hadoop/core-site.xml`**
```
<configuration>
    <property>
        <name>fs.defaultFS</name>
        <value>hdfs://localhost:9000</value>
    </property>
</configuration>
```

**`etc/hadoop/hdfs-site.xml`**
```
<configuration>
    <property>
        <name>dfs.replication</name>
        <value>1</value>
    </property>
</configuration>
```

In [13]:
%%bash
echo -e "<configuration> \n\
    <property> \n\
        <name>fs.defaultFS</name> \n\
        <value>hdfs://localhost:9000</value> \n\
    </property> \n\
</configuration>" >hadoop-3.4.0/etc/hadoop/core-site.xml

echo -e "<configuration> \n\
    <property> \n\
        <name>dfs.replication</name> \n\
        <value>1</value> \n\
    </property> \n\
</configuration>" >hadoop-3.4.0/etc/hadoop/hdfs-site.xml

Check

In [14]:
cat hadoop-3.4.0/etc/hadoop/hdfs-site.xml

<configuration> 
    <property> 
        <name>dfs.replication</name> 
        <value>1</value> 
    </property> 
</configuration>


## Set environment variables

Add the following lines to the Hadoop configuration script `hadoop-env.sh`(the script is in `hadoop-3.4.0/sbin`).
```
export HDFS_NAMENODE_USER=root
export HDFS_DATANODE_USER=root
export HDFS_SECONDARYNAMENODE_USER=root
export YARN_RESOURCEMANAGER_USER=root
export YARN_NODEMANAGER_USER=root
```

In [15]:
%%bash
cp -n hadoop-3.4.0/etc/hadoop/hadoop-env.sh hadoop-3.4.0/etc/hadoop/hadoop-env.sh.org
cat <<😃 >hadoop-3.4.0/etc/hadoop/hadoop-env.sh
export HDFS_NAMENODE_USER=root
export HDFS_DATANODE_USER=root
export HDFS_SECONDARYNAMENODE_USER=root
export YARN_RESOURCEMANAGER_USER=root
export YARN_NODEMANAGER_USER=root
😃

cp: warning: behavior of -n is non-portable and may change in future; use --update=none instead


## Setup localhost access via SSH key

We are going to allow passphraseless access to `localhost` with a secure key.

SSH must be installed and sshd must be running in order to use the Hadoop scripts that manage remote Hadoop daemons.



### Install `openssh` and start server

I'm not sure why we need the option `StrictHostKeyChecking no`. This option tells the `ssh` server to allow key authentication only from known hosts, in particular it prevents a host from authenticating with key if the key has changed. I guess this option is needed since a new ssh key is generated every time one runs this notebook.

Alternatively, one could just delete the file `~/.ssh/known_hosts` or else use `ssh-keygen -R hostname` to remove all keys belonging to hostname from the `known_hosts` file (see for instance [How to remove strict RSA key checking in SSH and what's the problem here?](https://serverfault.com/questions/6233/how-to-remove-strict-rsa-key-checking-in-ssh-and-whats-the-problem-here) or [Remove key from known_hosts](https://superuser.com/questions/30087/remove-key-from-known-hosts)). The option `ssh-keygen -R hostname` would be the most appropriate in a production setting where the file `~/.ssh/known_hosts` might contain other entries that you do not want to delete.


In [16]:
%%bash
apt-get update
apt-get -y install openssh-server
echo 'StrictHostKeyChecking no' >> /etc/ssh/ssh_config
/etc/init.d/ssh restart

Get:1 file:/etc/apt/apt-mirrors.txt Mirrorlist [144 B]


Hit:2 http://azure.archive.ubuntu.com/ubuntu noble InRelease


Hit:6 https://packages.microsoft.com/repos/azure-cli noble InRelease


Hit:7 https://packages.microsoft.com/ubuntu/24.04/prod noble InRelease


Hit:3 http://azure.archive.ubuntu.com/ubuntu noble-updates InRelease


...

Building dependency tree...


Reading state information...


openssh-server is already the newest version (1:9.6p1-3ubuntu13.12).


0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.


Restarting ssh (via systemctl): ssh.service.


### Generate key
Generate an SSH key that does not require a password.

The private key is contained in the file `id_rsa` located in the folder `~/.ssh`.

The public key is added to the file `~/.ssh/authorized_keys` in order to allow authentication with that key.

In [17]:
%%bash
rm $HOME/.ssh/id_rsa
ssh-keygen -t rsa -P '' -f ~/.ssh/id_rsa
cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys
chmod 0600 ~/.ssh/authorized_keys

rm: cannot remove '/root/.ssh/id_rsa': No such file or directory


Generating public/private rsa key pair.


Your identification has been saved in /root/.ssh/id_rsa


Your public key has been saved in /root/.ssh/id_rsa.pub


The key fingerprint is:


...

|   + o = o .     |


|  o . o o .      |


|  .o . o         |


|  .o..o          |


+----[SHA256]-----+


### Check SSH connection to localhost

The following command should output "hi!" if the connection works.

In [18]:
!ssh localhost "echo hi 👋"

hi 👋


# Launch a single-node Hadoop cluster

## Initialize the namenode

In [19]:
!hdfs namenode -format -nonInteractive

2025-07-12 14:56:21,683 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = fv-az1719-822/10.1.0.58
STARTUP_MSG:   args = [-format, -nonInteractive]
STARTUP_MSG:   version = 3.4.0
STARTUP_MSG:   classpath = /home/runner/work/big_data/big_data/hadoop-3.4.0/etc/hadoop:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/jetty-security-9.4.53.v20231009.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/kerb-common-2.0.3.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/listenablefuture-9999.0-empty-to-avoid-conflict-with-guava.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/netty-all-4.1.100.Final.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/commons-compress-1.24.0.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/common/lib/jakarta.activatio

r:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/animal-sniffer-annotations-1.17.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/jaxb-api-2.2.11.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/netty-resolver-dns-native-macos-4.1.100.Final-osx-x86_64.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/guava-27.0-jre.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/kerb-identity-2.0.3.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/gson-2.9.0.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/zookeeper-3.8.3.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/netty-buffer-4.1.100.Final.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/commons-configuration2-2.8.0.jar:/home/runner/work/big_data/big_data/hadoop-3.4.0/share/hadoop/hdfs/lib/jersey-core-1.19.4.jar:/home/

2025-07-12 14:56:21,812 INFO namenode.NameNode: createNameNode [-format, -nonInteractive]


2025-07-12 14:56:23,233 INFO namenode.NameNode: Formatting using clusterid: CID-9633ae23-f315-49be-af65-a073f3706c03
2025-07-12 14:56:23,253 INFO namenode.FSEditLog: Edit logging is async:true


2025-07-12 14:56:23,281 INFO namenode.FSNamesystem: KeyProvider: null
2025-07-12 14:56:23,282 INFO namenode.FSNamesystem: fsLock is fair: true
2025-07-12 14:56:23,282 INFO namenode.FSNamesystem: Detailed lock hold time metrics enabled: false


...

2025-07-12 14:56:23,618 INFO metrics.TopMetrics: NNTop conf: dfs.namenode.top.window.num.buckets = 10
2025-07-12 14:56:23,618 INFO metrics.TopMetrics: NNTop conf: dfs.namenode.top.num.users = 10
2025-07-12 14:56:23,618 INFO metrics.TopMetrics: NNTop conf: dfs.namenode.top.windows.minutes = 1,5,25
2025-07-12 14:56:23,621 INFO namenode.FSNamesystem: Retry cache on namenode is enabled
2025-07-12 14:56:23,621 INFO namenode.FSNamesystem: Retry cache will use 0.03 of total heap and retry cache entry expiry time is 600000 millis
2025-07-12 14:56:23,622 INFO util.GSet: Computing capacity for map NameNodeRetryCache
2025-07-12 14:56:23,622 INFO util.GSet: VM type       = 64-bit
2025-07-12 14:56:23,623 INFO util.GSet: 0.029999999329447746% max memory 3.9 GB = 1.2 MB
2025-07-12 14:56:23,623 INFO util.GSet: capacity      = 2^17 = 131072 entries
2025-07-12 14:56:23,645 INFO namenode.FSImage: Allocated new BlockPoolId: BP-1021452147-10.1.0.58-1752332183640


2025-07-12 14:56:23,707 INFO common.Storage: Storage directory /tmp/hadoop-root/dfs/name has been successfully formatted.


2025-07-12 14:56:23,746 INFO namenode.FSImageFormatProtobuf: Saving image file /tmp/hadoop-root/dfs/name/current/fsimage.ckpt_0000000000000000000 using no compression


2025-07-12 14:56:23,819 INFO namenode.FSImageFormatProtobuf: Image file /tmp/hadoop-root/dfs/name/current/fsimage.ckpt_0000000000000000000 of size 399 bytes saved in 0 seconds .
2025-07-12 14:56:23,832 INFO namenode.NNStorageRetentionManager: Going to retain 1 images with txid >= 0
2025-07-12 14:56:23,835 INFO blockmanagement.DatanodeManager: Slow peers collection thread shutdown
2025-07-12 14:56:23,855 INFO namenode.FSNamesystem: Stopping services started for active state
2025-07-12 14:56:23,855 INFO namenode.FSNamesystem: Stopping services started for standby state


2025-07-12 14:56:23,869 INFO namenode.FSImage: FSImageSaver clean checkpoint: txid=0 when meet shutdown.
2025-07-12 14:56:23,869 INFO namenode.NameNode: SHUTDOWN_MSG: 
/************************************************************
SHUTDOWN_MSG: Shutting down NameNode at fv-az1719-822/10.1.0.58
************************************************************/


## Start cluster

In [20]:
!$HADOOP_HOME/sbin/start-dfs.sh

Starting namenodes on [localhost]


Starting datanodes


Starting secondary namenodes [fv-az1719-822]


In [21]:
%%bash
# Check if HDFS is in safe mode
if hdfs dfsadmin -safemode get | grep 'ON'; then
  echo "Namenode is in safe mode. Leaving safe mode..."
  hdfs dfsadmin -safemode leave
else
  echo "Namenode is not in safe mode."
fi

Namenode is not in safe mode.


# Run some simple HDFS commands

In [22]:
%%bash
# create directory "my_dir" in HDFS home
hdfs dfs -mkdir /user
hdfs dfs -mkdir /user/root # this is the "home" of user root on HDFS
hdfs dfs -mkdir my_dir

# if sampls_data does not exist, create it (so that the notebook can run also outside of Colab)
mkdir -p sample_data
touch sample_data/mnist_test.csv

# Check if the file is empty and fill it if needed
if [ ! -s sample_data/mnist_test.csv ]; then
  echo -e "0 1 2 3 4\n5 6 7 8 9" > sample_data/mnist_test.csv
fi


# upload file mnist_test.csv to my_dir
hdfs dfs -put sample_data/mnist_test.csv my_dir/

# show contents of directory my_dir
hdfs dfs -ls -h my_dir

Found 1 items


-rw-r--r--   1 root supergroup         20 2025-07-12 14:56 my_dir/mnist_test.csv


# Run some simple MapReduce jobs

We'll employ the [streaming](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html) library, which broadens our options by enabling the use of any programming language for both the mapper and/or the reducer.

With this utility any executable or file containing code that the operating system can interpret and execute directly, can serve as mapper and/or reducer.

## Simplest MapReduce job

This is a "no-code" example since we are going to use the existing Unix commands `cat` and `wc` respectively as mapper and as reducer. The result will show a line with three values: the counts of lines, words, and characters in the input file(s).

Input folder is `/user/my_user/my_dir/`, output folder `/user/my_user/output_simplest`.

**Note**: the output folder should not exist because it is created by Hadoop (this is in accordance with Hadoop's principle of not overwriting data).

Now run the MapReduce job

In [23]:
%%bash

hdfs dfs -rm -r output_simplest || hdfs namenode -format -nonInteractive
mapred streaming \
  -input my_dir \
  -output output_simplest \
  -mapper /bin/cat \
  -reducer /usr/bin/wc

rm: `output_simplest': No such file or directory


namenode is running as process 3373.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.


2025-07-12 14:56:46,096 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties


2025-07-12 14:56:46,181 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).


2025-07-12 14:56:46,181 INFO impl.MetricsSystemImpl: JobTracker metrics system started


...

	File Input Format Counters 


		Bytes Read=20


	File Output Format Counters 


		Bytes Written=25


2025-07-12 14:56:49,930 INFO streaming.StreamJob: Output directory: output_simplest


If the `output` directory contains the empty file `_SUCCESS`, this means that the job was successful.

Check the output of the MapReduce job.

In [24]:
!hdfs dfs -cat output_simplest/part-00000

      2      10      22	


The number of words is in this case equal to the number of lines because there are no word separators (empty spaces) in the file, so each line is a word.

## Another MapReduce example: filter a log file

We're going to use a Linux logfile and look for the string `sshd` in a given position. The file stems from [Loghub](https://github.com/logpai/loghub), a freely available collection of system logs for AI-driven log analytics research.

The mapper `mapper.py` filters the file for the given string `sshd` at field 4.

The job has no reducer (option `-reducer NONE`). Note that without a reducer the sorting and shuffling phase after the map phase is skipped.


Download the logfile `Linux_2k.log`:

In [25]:
!wget --no-clobber https://raw.githubusercontent.com/logpai/loghub/master/Linux/Linux_2k.log

--2025-07-12 14:56:51--  https://raw.githubusercontent.com/logpai/loghub/master/Linux/Linux_2k.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 

200 OK
Length: 216485 (211K) [text/plain]
Saving to: ‘Linux_2k.log’

Linux_2k.log        100%[===================>] 211.41K  --.-KB/s    in 0.01s   

2025-07-12 14:56:52 (17.2 MB/s) - ‘Linux_2k.log’ saved [216485/216485]



In [26]:
%%bash
hdfs dfs -mkdir input || true
hdfs dfs -put Linux_2k.log input/ || true

Define the mapper

In [27]:
%%writefile mapper.py
#!/usr/bin/env python
import sys

for line in sys.stdin:
    # split the line into words
    line = line.strip()
    fields = line.split()
    if (len(fields)>=5 and fields[4].startswith('sshd')):
      print(line)


Writing mapper.py


Test the script (after setting the correct permissions)

In [28]:
!chmod 700 mapper.py

Look at the first 10 lines

In [29]:
!head -10 Linux_2k.log

Test the mapper in the shell (not using MapReduce):

In [30]:
!head -100 Linux_2k.log| ./mapper.py

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root
Jun 15 02:04:59 combo sshd(p

Now run the MapReduce job on the pseudo-cluster

In [31]:
%%bash

hdfs dfs -rm -r output_filter

mapred streaming \
  -file mapper.py \
  -input input \
  -output output_filter \
  -mapper mapper.py \
  -reducer NONE


rm: `output_filter': No such file or directory


2025-07-12 14:56:57,233 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.


packageJobJar: [mapper.py] [] /tmp/streamjob1385876954389327095.jar tmpDir=null


2025-07-12 14:56:57,787 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties


2025-07-12 14:56:57,863 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).


...

	File Input Format Counters 


		Bytes Read=216485


	File Output Format Counters 


		Bytes Written=85436


2025-07-12 14:56:59,470 INFO streaming.StreamJob: Output directory: output_filter


Check the result

In [32]:
!hdfs dfs -ls output_filter

Found 2 items
-rw-r--r--   1 root supergroup          0 2025-07-12 14:56 output_filter/_SUCCESS
-rw-r--r--   1 root supergroup      85436 2025-07-12 14:56 output_filter/part-00000


In [33]:
!hdfs dfs -cat output_filter/part-00000 |head

Jun 14 15:16:01 combo sshd(pam_unix)[19939]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: check pass; user unknown	
Jun 14 15:16:02 combo sshd(pam_unix)[19937]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=218.188.2.4	
Jun 15 02:04:59 combo sshd(pam_unix)[20882]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20884]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20883]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo sshd(pam_unix)[20885]: authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=220-135-151-1.hinet-ip.hinet.net  user=root	
Jun 15 02:04:59 combo

## Aggregate data with MapReduce

Following the example in [Hadoop Streaming/Aggregate package](https://hadoop.apache.org/docs/stable/hadoop-streaming/HadoopStreaming.html#Hadoop_Aggregate_Package)

In [34]:
%%writefile myAggregatorForKeyCount.py
#!/usr/bin/env python
import sys

def generateLongCountToken(id):
    return "LongValueSum:" + id + "\t" + "1"

def main(argv):
    line = sys.stdin.readline()
    try:
        while line:
            line = line[:-1]
            fields = line.split()
            s = fields[4].split('[')[0]
            print(generateLongCountToken(s))
            line = sys.stdin.readline()
    except "end of file":
        return None

if __name__ == "__main__":
     main(sys.argv)

Writing myAggregatorForKeyCount.py


Set permissions

In [35]:
!chmod 700 myAggregatorForKeyCount.py

Test the mapper

In [36]:
!head -20 Linux_2k.log| ./myAggregatorForKeyCount.py

LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:logrotate:	1
LongValueSum:su(pam_unix)	1
LongValueSum:su(pam_unix)	1
LongValueSum:sshd(pam_unix)	1
LongValueSum:sshd(pam_unix)	1


Run the MapReduce job

In [37]:
%%bash

chmod +x myAggregatorForKeyCount.py

hdfs dfs -rm -r output_aggregate

mapred streaming \
  -input input \
  -output output_aggregate \
  -mapper myAggregatorForKeyCount.py \
  -reducer aggregate \
  -file myAggregatorForKeyCount.py


rm: `output_aggregate': No such file or directory


2025-07-12 14:57:05,125 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.


packageJobJar: [myAggregatorForKeyCount.py] [] /tmp/streamjob1220146221685080510.jar tmpDir=null


2025-07-12 14:57:05,744 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties


2025-07-12 14:57:05,824 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).


...

	File Input Format Counters 


		Bytes Read=216485


	File Output Format Counters 


		Bytes Written=326


2025-07-12 14:57:07,391 INFO streaming.StreamJob: Output directory: output_aggregate


Check result

In [38]:
%%bash
hdfs dfs -ls output_aggregate
hdfs dfs -cat output_aggregate/part-00000

Found 2 items


-rw-r--r--   1 root supergroup          0 2025-07-12 14:57 output_aggregate/_SUCCESS


-rw-r--r--   1 root supergroup        326 2025-07-12 14:57 output_aggregate/part-00000


--	1


bluetooth:	2


...

sysctl:	1


syslog:	2


syslogd	7


udev	8


xinetd	2


Pretty-print table of aggregated data

In [39]:
%%bash
hdfs dfs -get output_aggregate/part-00000 result # download results file
# Use awk to format the output into columns and then sort by the second field numerically in descending order
awk '{printf "%-20s %s\n", $1, $2}' result | sort -k2nr

ftpd                 916


sshd(pam_unix)       677


su(pam_unix)         172


kernel:              76


klogind              46


...

rpc.statd            1


rpcidmapd:           1


sdpd                 1


snmpd                1


sysctl:              1


# Stop cluster

When you're done with your computations, you can shut down the Hadoop cluster and stop the `sshd` service.

In [40]:
!./hadoop-3.4.0/sbin/stop-dfs.sh

Stopping namenodes on [localhost]


Stopping datanodes


Stopping secondary namenodes [fv-az1719-822]


Stop the `sshd` daemon

In [41]:
!/etc/init.d/ssh stop



.


# Concluding remarks

We have started a single-node Hadoop cluster and ran some simple HDFS and MapReduce commands.

Even when running on a single machine, one can benefit from the parallelism provided by multiple virtual cores.

Hadoop provides also a command-line utility (the CLI MiniCluster) to start and stop a single-node Hadoop cluster "_without the need to set any environment variables or manage configuration files_" (https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/CLIMiniCluster.html). The [Hadoop MiniCluster](https://github.com/groda/big_data/blob/master/Hadoop_minicluster.ipynb) notebook serves as a guide for launching the Hadoop MiniCluster.

While it can be useful to be able to start a Hadoop cluster with a single command, delving into the functionality of each component offers valuable insights into the intricacies of Hadoop architecture, thereby enriching the learning process.

If you found this notebook helpful, consider exploring:
 - [Hadoop single-node cluster setup with Python](https://github.com/groda/big_data/blob/master/Hadoop_single_node_cluster_setup_Python.ipynb) similar to this but using Python in place of bash
 - [Setting up Spark Standalone on Google Colab](https://github.com/groda/big_data/blob/master/Hadoop_Setting_up_Spark_Standalone_on_Google_Colab.ipynb)
 - [Getting to know the Spark Standalone Architecture](https://github.com/groda/big_data/blob/master/Spark_Standalone_Architecture_on_Google_Colab.ipynb)


